CAPÍTULO 9 - Apply
-


Conhecer apply é um conceito fundamental no processo de limpeza de dados. Apply aceita uma função e execulta em cada linha ou coluna do dataframe simultamneamente.

9.3 Apply (básico)

In [116]:
import pandas as pd

df = pd.DataFrame({'a': [10,20,30],
                   'b': [20,30,40]})

print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [117]:
def my_sq(x):
    return x ** 2

Podemos aplicar funções com apply em uma Series(isto é, uma coluna ou linha individual)

In [118]:
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


9.3.1 apply em uma Series

Em nosso exemplo, se o subconjunto for composto por uma só coluna ou linha, o seu type será uma Series do Pandas

Series tem um método chamado apply, para usá-lo, devemos passar a função que queremos usar em cada elemento da Series.

In [119]:
sq = df['a'].apply(my_sq)

print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


Observe que não precisamos de parênteses quando passamos a função

Vamos fazer um outro exemplo mas agora com uma função que use dois parâmetros

In [120]:
def my_exp(x, e):
    return x ** e

Se quisermos usar a função devos passar um segundo parâmetro. Para isso basta passar o segundo argumento como uma Keyword Argument (argumento nomeado) para apply:

In [121]:
ex = df['a'].apply(my_exp, e=3)

print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


9.3.2 apply em um DataFrame

In [122]:
print(df)

    a   b
0  10  20
1  20  30
2  30  40


DataFrames têm pelo menos duas dimensões, então devemos expecificar em qual eixo em que a função será aplicada

In [123]:
def printMe(x):
    print(x)

9.3.2.1 Operações em colunas

Use o parâmetro axis=0 em apply para trabalhar com colunas

In [124]:
df.apply(printMe, axis=0)

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


a    None
b    None
dtype: object

Quando aplicamos uma função em um DataFrame, o eixo todo é passado como primeiro argumento da função

Para ilustrar melhor vamos escrever uma função que calcule a média de três números

In [125]:
def avg3(x, y, z):
    return (x + y + z) / 3

Se tentarmos aplicar essa função em nossar colunas veremos um erro

In [126]:
"""
print(df.apply(avg3))
"""

'\nprint(df.apply(avg3))\n'

A função recebe o primeiro argumento, mas deixa de receber y e z, isso porque quando usamos apply a coluna toda é passada como primeiro argumento

Para que essa função seja apropriada ao método apply, será nescessário reescrever algumas partes

In [127]:
def avg3Apply(cool):
    x = cool[0]
    y = cool[1]
    z = cool[2]

    return (x + y + z) / 3

In [128]:
print(df.apply(avg3Apply))

a    20.0
b    30.0
dtype: float64


9.3.2.2 Operações em linhas

As operações em linhas funcionam como as operações me colunas, com a diferença que agora passamos axis=1,  
ao invés de a primeira coluna interira ser passado como primeiro parâmetro, agora a primeira linha inteira é passada como primeiro parâmetro

No entanto, a função que escrevemos não funcionará para as linhas, poís temos apenas 2 números em uma linha, e a função precisa de 3 para funcionar

In [129]:
"""
print(df.apply(avg3Apply, axis=1))
""" 

'\nprint(df.apply(avg3Apply, axis=1))\n'

In [130]:
def avg2Apply(row):
    x = row[0]
    y = row[1]

    return (x + y) / 2

In [131]:
print(df.apply(avg2Apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


C:\Users\User\AppData\Local\Temp\ipykernel_23476\2954858172.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = row[0]
C:\Users\User\AppData\Local\Temp\ipykernel_23476\2954858172.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = row[1]


9.4 apply (mais avançado)

A biblioteca seaborn tem um conjunto de dados embutido chamdo titanic que informa se algum indivíduo sobreviveu ao naufrágio do Titanic

In [132]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [133]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


Uma forma de usar apply é calcular quatos valores null ou NaN há em nossos dados, assim como o percentual de casos completos em cada coluna ou cada linha.  
Vamos escrever algumas funções

Vec = Vector
-


- Número de valores ausentes

In [134]:
import numpy as np

def count_missing(vec):

    #obtém um vetor com valores True/Flase conforme o valor estiver ausente ou não
    nullVec = pd.isnull(vec)

    #soma todos os valore que forem nulos
    nullCont = sum(nullVec)

    return nullCont

- Proporção de valors ausentes

In [135]:
def prop_missing(vec):

    #numerador: número de valres ausentes
    num = count_missing(vec)

    #denominador: número total de valores em um vetor
    #precisamos contabilizar também os valores ausentes
    dem = vec.size

    return num / dem

- Proporção de valores completos

In [136]:
#porcentagem de valores não ausentes
def prop_complete(vec):

    return 1 - prop_missing(vec)

9.4.1 Operações em coluna 

Vamo aplicar nossas funções me cada coluna dos nossos dados

In [137]:
cmis_col = titanic.apply(count_missing)
pmis_col = titanic.apply(prop_missing)
pcom_col = titanic.apply(prop_complete)

print(cmis_col, '\n')
print(pmis_col, '\n')
print(pcom_col, '\n')

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64 

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64 

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive      

Como temos contadores de valores ausentes podemos determinar se uma coluna é ou não uma opção viável para análise.  
Exemplo, na coluna embark_town, podemos ver que somente dois valores estão ausentes, e podemos verificar se estão assim aleatoriamente ou se há algum motivo especial para isso

In [138]:
print(titanic.loc[pd.isnull(titanic.embark_town), :])

     survived  pclass     sex   age  sibsp  parch  fare embarked  class  \
61          1       1  female  38.0      0      0  80.0      NaN  First   
829         1       1  female  62.0      0      0  80.0      NaN  First   

       who  adult_male deck embark_town alive  alone  
61   woman       False    B         NaN   yes   True  
829  woman       False    B         NaN   yes   True  


Outra observação é que a variável deck em 688 (77,2%) de valores ausentes. A menos que tenha mais investigações, é seguro afirmar que esses dados não são úteis para analise

9.4.2 Operações em linhas

Como as funções são vetorizadas podemos passar nas linhas sem alterá-las

In [139]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row, '\n')
print(pmis_row, '\n')
print(pcom_row, '\n')

0      1
1      0
2      1
3      0
4      1
      ..
886    1
887    0
888    2
889    0
890    1
Length: 891, dtype: int64 

0      0.066667
1      0.000000
2      0.066667
3      0.000000
4      0.066667
         ...   
886    0.066667
887    0.000000
888    0.133333
889    0.000000
890    0.066667
Length: 891, dtype: float64 

0      0.933333
1      1.000000
2      0.933333
3      1.000000
4      0.933333
         ...   
886    0.933333
887    1.000000
888    0.866667
889    1.000000
890    0.933333
Length: 891, dtype: float64 



Uma tarefa que podemos fazer é verificar se há alguma linha em nossos dados que tenha diversos valores ausentes

In [140]:
print(cmis_row.value_counts())

1    549
0    182
2    160
Name: count, dtype: int64


Como estamos usando apply podemos criar uma nova coluna contendo esses valores

In [141]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)

print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [142]:
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
485         0       3  female   NaN      3      1  25.4667        S  Third   
680         0       3  female   NaN      0      0   8.1375        Q  Third   
760         0       3    male   NaN      0      0  14.5000        S  Third   
878         0       3    male   NaN      0      0   7.8958        S  Third   
274         1       3  female   NaN      0      0   7.7500        Q  Third   
250         0       3    male   NaN      0      0   7.2500        S  Third   
198         1       3  female   NaN      0      0   7.7500        Q  Third   
522         0       3    male   NaN      0      0   7.2250        C  Third   
829         1       1  female  62.0      0      0  80.0000      NaN  First   
502         0       3  female   NaN      0      0   7.6292        Q  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
485  woman       False  NaN  Southampton    no  False            2  
680

9.5 Funções vetorizadas

Quando usamos apply podemos fazer uma função atuar coluna a coluna ou linha a linha, porém as vezes será necessario reescrever a função. No entanto nem sempre será viável reescrever uma função inteira, mas podemos tirar proveito da função vectorize e de um decorador para vetorizar qualquer função

In [143]:
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [144]:
def avg2(x, y):
    return (x+y) / 2

In [145]:
print(avg2(df['a'], df['b']))

0    15.0
1    25.0
2    35.0
dtype: float64


Essa abordagem funciona porque os cáculos propriamente ditos da nossa função são inerentemente vetorizados. Ou seja se somarmos duas colunas numéricas o Pandas e o numpy fará a soma entre os elementos de modo automático

Vamos modificar a função e fazer um cálculo não vetorizado

In [146]:
import numpy as np

def avg2Mod(x, y):
    """
    Calcula a média a menos que x seja 20
    """
    if (x == 20):
        return(np.nan)
    else:
        return (x + y) / 2

Se executada passar uma Series a função causará erro

In [147]:
"""
print(avg2Mod(df['a'], df['b']))
"""

"\nprint(avg2Mod(df['a'], df['b']))\n"

In [148]:
print(avg2Mod(10, 20))
print(avg2Mod(20, 20))

15.0
nan


9.5.1 Usando o numpy

Queremos alterar nossa função de modo que ao receber um vetor, ela faça os cálculos elemento a elemento. Passar isso podemos usar a função vectorize do numpy

In [149]:
avg2Mod_vec = np.vectorize(avg2Mod)

print(avg2Mod_vec(df['a'], df['b']))

[15. nan 35.]


Esse método funcionará bem se você não tiver o código-fonte de uma função. No entanto se tiver escrevendo a própria função  poderá usar um decorador Python para vetorizar "automaticamente"

In [150]:
#para usar o decorador vectorize usamos @ antes da nossa def function
@np.vectorize
def v_avg_2_mod(x, y):
    if (x == 20):
        return(np.NaN)
    else:
        return (x+y)/2

In [151]:
#podemos usar diretamente a função vetorizada sem criar uma nova

print(v_avg_2_mod(df['a'], df['b']))

[15. nan 35.]


9.5.2 Usando a biblioteca numba

A biblioteca numba tem o mesmo decorador vectorize que a numpy, e a usa da mesma forma: @ numba.vectorize

9.6 Funções lambda

As vezes a função usado no método apply é tão simples que não haverá necessidade de criar uma função separada.

In [157]:
import regex 

docs = pd.read_csv('../../data/doctors.csv', header=None)

print(docs)

                               0
0     William Hartnell (1963-66)
1    Patrick Troughton (1966-69)
2          Jon Pertwee (1970 74)
3            Tom Baker (1974-81)
4        Peter Davison (1982-84)
5          Colin Baker (1984-86)
6      Sylvester McCoy (1987-89)
7             Paul McGann (1996)
8   Christopher Eccleston (2005)
9        David Tennant (2005-10)
10          Matt Smith (2010-13)
11     Peter Capaldi (2014-2017)
12        Jodie Whittaker (2017)


In [159]:
p = regex.compile(r'\w+\s+\w+')

def get_name(s):
    return p.match(s).group()

docs['name_func'] = docs[0].apply(get_name)

In [160]:
print(docs)

                               0              name_func
0     William Hartnell (1963-66)       William Hartnell
1    Patrick Troughton (1966-69)      Patrick Troughton
2          Jon Pertwee (1970 74)            Jon Pertwee
3            Tom Baker (1974-81)              Tom Baker
4        Peter Davison (1982-84)          Peter Davison
5          Colin Baker (1984-86)            Colin Baker
6      Sylvester McCoy (1987-89)        Sylvester McCoy
7             Paul McGann (1996)            Paul McGann
8   Christopher Eccleston (2005)  Christopher Eccleston
9        David Tennant (2005-10)          David Tennant
10          Matt Smith (2010-13)             Matt Smith
11     Peter Capaldi (2014-2017)          Peter Capaldi
12        Jodie Whittaker (2017)        Jodie Whittaker


Podemos ver que está é uma função muito simples de uma única linha. Em geral quando isso acontece podemos optar por escrevê-la diretamente no método apply,   
essa abordagem é conhecida como o uso de funções lambda.

In [162]:
docs['name_lamb'] = docs[0].apply(lambda x: p.match(x).group())
print(docs)

                               0              name_func              name_lamb
0     William Hartnell (1963-66)       William Hartnell       William Hartnell
1    Patrick Troughton (1966-69)      Patrick Troughton      Patrick Troughton
2          Jon Pertwee (1970 74)            Jon Pertwee            Jon Pertwee
3            Tom Baker (1974-81)              Tom Baker              Tom Baker
4        Peter Davison (1982-84)          Peter Davison          Peter Davison
5          Colin Baker (1984-86)            Colin Baker            Colin Baker
6      Sylvester McCoy (1987-89)        Sylvester McCoy        Sylvester McCoy
7             Paul McGann (1996)            Paul McGann            Paul McGann
8   Christopher Eccleston (2005)  Christopher Eccleston  Christopher Eccleston
9        David Tennant (2005-10)          David Tennant          David Tennant
10          Matt Smith (2010-13)             Matt Smith             Matt Smith
11     Peter Capaldi (2014-2017)          Peter Capa

Funções lambda são úteis para escrever códigos simples, embora seja possível escrever funções complexas, não é muito recomendável que o faça